# Building a demo for the PyTorch model you just trained

In this notebook, you'll build a complete webapp with the pytorch model you trained running in the backend (which is either a google GPU server or your laptop based on what you chose for the runtime). Frontend will be simple HTML mixed with some javascript (to talk to the python backend) and CSS (to make things look a bit pretty). You'll able to do all of this in few simple steps! in few minutes!

At the end of the tutorial, you'll be running an app that looks like this

![final_demo](https://github.com/kampta/AI4ALL/blob/master/photag/imgs/action.png?raw=true)



* Start with making a copy of this notebook first!
* Open the notebook in Colab and choose **Connect to a hosted runtime** on top left (if you want to run things on Google's servers). Something like this

![runtime](https://github.com/kampta/AI4ALL/blob/master/photag/imgs/runtime.png?raw=true)



## Boileplate Code

We will start with installing some dependencies and importing some boilerplate code

In [ ]:
!pip install flask-ngrok
!git clone https://github.com/kampta/AI4ALL.git
%cd AI4ALL/photag

Cloning into 'AI4ALL'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 24 (delta 3), reused 23 (delta 2), pack-reused 0
Unpacking objects: 100% (24/24), done.
/content/AI4ALL/photag


In [ ]:
import traceback
from flask import Flask, jsonify, request, render_template
from flask_ngrok import run_with_ngrok

from utils import read_file, transform_image, get_topk, render_prediction

## Define the app endpoints

Endpoints are part of the URL that are defined to access some specific things from the website. You don't need to worry about them too much for now 

In [ ]:
app = Flask(__name__)


@app.route('/', methods=['GET'])
def demo_photag():
    return render_template('photag.html')


@app.route('/predict', methods=['GET', 'POST'])
def predict():
    if request.method == 'GET':
        try:
            url = request.args.get('q')
            app.logger.debug('url provided - %s', url)
            input_tensor = transform_image(read_file(url=url))
            values, indices = get_topk(input_tensor)
            results = render_prediction(values, indices)
            return jsonify(results=results)

        except:
            app.logger.debug("Error: %s", traceback.print_exc())
            return jsonify("invalid image url")

    elif request.method == 'POST':
        try:
            file = request.files['file']
            app.logger.debug('file uploaded - %s', file)
            url = request.form.get("url", None)
            app.logger.debug('url provided - %s', url)

            input_tensor = transform_image(read_file(upload=file, url=url))
            values, indices = get_topk(input_tensor)
            results = render_prediction(values, indices)
            return jsonify(results=results)

        except:
            app.logger.debug("Error: %s", traceback.print_exc())
            return jsonify("invalid image")

    else:
        app.logger.debug("Error: %s", traceback.print_exc())
        return jsonify('invalid request')


## Running the app

That's it! Just run the app with following code. Wait for an ngrok URL to show up. It would look something like this - "http://a1b2c3d4e5.ngrok.io". Click on the URL and you should be able to access your web app.



In [ ]:
run_with_ngrok(app)   #starts ngrok when the app is run
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://0a6feb07a050.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [12/Jul/2020 00:46:55] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 00:46:56] "GET /static/main.css HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 00:46:56] "GET /static/main.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Jul/2020 00:46:57] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [12/Jul/2020 00:47:04] "POST /predict HTTP/1.1" 200 -


Great job! You've built your first web service that uses a 
pytorch model at the backend to classify images in the wild


---




*Any feedback for this tutorial?* Please shoot me an email at kampta@umd.edu